# quantile_data.ipynb
Requirement: Add a column named Distance_quantile to the data with the value of distance between Origin and Destination.
If the the Distance is within the quantile of [0.05, 0.95], set the distance_quantile with value of distance;
if the the Distance is outside the quantile of [0.05, 0.95], set the distance_quantile with value of 0.

For example:

In the following df, we want the value of Distance_quantile to be 1 for FlightNum 0 to 3; 
we want the value of Distance_quantile to be 0 for FlightNum 4; 
we want the value of Distance_quantile to be 10 for FlightNum 5 to 8; 
we want the value of Distance_quantile to be 0 for FlightNum 9; 
we want the value of Distance_quantile to be 20 for FlightNum 10 to 13;
we want the value of Distance_quantile to be 0 for FlightNum 14. 


In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({'FlightNum': range(15),
                    'Origin':      ['A','A','A','A','A','B','B','B','B','B','C','C','C','C','C'],
                    'Destination': ['M','M','M','M','M','N','N','N','N','N','H','H','H','H','H'],
                    'Distance':    [1,1,1,1,500, 10,10,10,10,2000, 20,20,20,20,3000],
                  })

In [3]:
df

,FlightNum,Origin,Destination,Distance
0,0,A,M,1
1,1,A,M,1
2,2,A,M,1
3,3,A,M,1
4,4,A,M,500
5,5,B,N,10
6,6,B,N,10
7,7,B,N,10
8,8,B,N,10
9,9,B,N,2000


In [4]:
#.05 quantile
def q05(x):
    return x.quantile(0.05)

#.95 quantile
def q95(x):
    return x.quantile(0.95)

#我们经常用IQR来去除数据
#IQR lower bound
def iqr_lower(x):
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    IQR = q3 - q1
    lower_range = q1 - 1.5 * IQR
    return lower_range

#IQR higher bound
def iqr_higher(x):
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    IQR = q3 - q1
    upper_range = q3 + 1.5 * IQR
    return upper_range

agg_table = df.groupby(['Origin','Destination']).agg({'Distance':['median', q05, q95, iqr_lower, iqr_higher]}).reset_index()
agg_table

Origin Destination Distance                                   
                       median   q05     q95 iqr_lower iqr_higher
0      A           M      1.0   1.0   400.2       1.0        1.0
1      B           N     10.0  10.0  1602.0      10.0       10.0
2      C           H     20.0  20.0  2404.0      20.0       20.0

In [5]:
results = []
for Origin, dest, lower, higher, median in zip(agg_table['Origin'], agg_table['Destination'], agg_table['Distance']['q05'], agg_table['Distance']['q95'], agg_table['Distance']['median']):
    #print(Origin, dest, lower, higher)
    df_filtered = df[(df['Origin'] == Origin) & (df['Destination'] == dest)].copy()
    # 可以用0，不过median更好
    df_filtered.loc[(df_filtered['Distance'] < lower) | (df_filtered['Distance'] > higher), 'Distance'] = median
    
    results.append(df_filtered) 

In [6]:
results[0].append(results[1]).append(results[2])

,FlightNum,Origin,Destination,Distance
0,0,A,M,1
1,1,A,M,1
2,2,A,M,1
3,3,A,M,1
4,4,A,M,1
5,5,B,N,10
6,6,B,N,10
7,7,B,N,10
8,8,B,N,10
9,9,B,N,10


### END